In [278]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import numpy as np
from io import BytesIO
from typing import *
import framework_functions

In [282]:
#table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")
table = pq.read_table(r"C:\Users\Cristiano Lavoro\Desktop\benchmarks\imagenette\imagenette2\output\ds.parquet")

In [270]:
print(draw_tree_schema(table.take([0]).to_pydict()))

└── image_feature
    └── [0]
        └── [0]
            ├── class_feature
            │   ├── [0]
            │   │   └── label
            ├── image
            └── text_feature
                └── [0]
                    └── text



In [276]:
batches = table.to_batches(300)
save_path = "imagenette/imagenette2/output/partitioning"

In [277]:
for i,batch in enumerate(batches):
    with pa.OSFile(f"{save_path}/ds_stream_{i+1}_of_{len(batches)}.arrows", 'wb') as sink:  
        with pa.ipc.new_stream(sink, batches[0].schema) as writer: 
            writer.write_batch(batch)  

In [ ]:
from datasets import Dataset

In [313]:
schema = table.schema
batches = [recordbatch for recordbatch in table.to_batches(300) if len(recordbatch) > 0]
offsets = np.cumsum([0] + [len(b) for b in batches], dtype=np.int64)

In [311]:
offset = 320
length = 1

In [308]:
def _interpolation_search(arr: List[int], x: int) -> int:
    """
    Return the position i of a sorted array so that arr[i] <= x < arr[i+1]

    Args:
        arr (`List[int]`): non-empty sorted list of integers
        x (`int`): query

    Returns:
        `int`: the position i so that arr[i] <= x < arr[i+1]

    Raises:
        `IndexError`: if the array is empty or if the query is outside the array values
    """
    i, j = 0, len(arr) - 1
    while i < j and arr[i] <= x < arr[j]:
        k = i + ((j - i) * (x - arr[i]) // (arr[j] - arr[i]))
        if arr[k] <= x < arr[k + 1]:
            return k
        elif arr[k] < x:
            i, j = k + 1, j
        else:
            i, j = i, k
    raise IndexError(f"Invalid query '{x}' for size {arr[-1] if len(arr) else 'none'}.")

In [ ]:
### Non ci interessa fare lo sclicing di un vettore di batch che, di fatto, è già in memory. Dato il vettore di 
### offset (creato e storato come metadato in fase di scrittura partizionata) dobbiamo aprire on the fly la tabella (o il batch, che dir si voglia)
### dato l'indice e l'informazione storata nell'offset

In [316]:
def stolen(offset,batches):
    if offset < 0:
        raise IndexError("Offset must be non-negative")
    elif offset >= offsets[-1] or (length is not None and length <= 0):
        return pa.Table.from_batches([], schema=schema)
    i = _interpolation_search(offsets, offset)
    if length is None or length + offset >= offsets[-1]:
        batches = batches[i:]
        batches[0] = batches[0].slice(offset - offsets[i])
    else:
        j = _interpolation_search(offsets, offset + length - 1)
        batches = batches[i : j + 1]
        batches[-1] = batches[-1].slice(0, offset + length - offsets[j])
        batches[0] = batches[0].slice(offset - offsets[i])
    return pa.Table.from_batches(batches, schema=schema)

In [318]:
len(stolen(offset,batches))

1